In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data3=pd.read_csv('../input/big-mart-sales-prediction/Train.csv')
data1=pd.read_csv('../input/big-mart-sales-prediction/Test.csv')
print(data3)
print(data1)

# LET'S UNDERSTAND TRAINING DATA****

CHECK FOR NULL VALUES

In [ ]:
data3.isnull().sum()

**AS WE CAN SEE THAT Item_Weight  AND Outlet_size CONTAINS NULL VALUES, WE NEED TO GET RID OF IT**

# STATISTICAL DESCRIPTION OF DATA****

In [ ]:
data3.describe()

In [ ]:
corr_matrix=data3.corr()
corr_matrix['Item_Outlet_Sales']

***WE CAN SEE Item_MRP IS CORRELATED TO Item_Outlet_Sales AND REST OTHERS DOES NOT MATTER MUCH***

In [ ]:
data3.Item_Fat_Content=data3.Item_Fat_Content.replace('LF','Low Fat')
data3.Item_Fat_Content=data3.Item_Fat_Content.replace('reg','Regular')
data3.Item_Fat_Content=data3.Item_Fat_Content.replace('low fat','Low Fat')


**THERE MUST BE SOME KIND OF TYPO IN tem_Fat_Content AS SAME THING IS WRITTEN IN DIFFERENT WAYS  **

**WE REPLACE NULL VALUES IN Item_Weight WITH MEAN AND Outlet_Size WITH MEDDIUM**

In [ ]:
data3['Item_Weight'].fillna(data3['Item_Weight'].mean(), inplace=True)

In [ ]:
data3['Outlet_Size']=data3['Outlet_Size'].replace(np.nan,'Medium' )

In [ ]:
data3.isnull().sum()

**SINCE ITEM WEIGHT IS HIGHLY CORRELATED WITH TARGET VARIABLE WE EXPLORE IT GRAPHICALLY**

In [ ]:
fig,axes=plt.subplots(1,1,figsize=(12,8))
sns.scatterplot(x='Item_MRP',y='Item_Outlet_Sales',hue='Item_Fat_Content',size='Item_Weight',data=data3)

**NOW WE EXPLORE OTHER COLUMNS**

In [ ]:
fig,axes=plt.subplots(3,1,figsize=(15,12))
sns.scatterplot(x='Item_Visibility',y='Item_Outlet_Sales',hue='Item_MRP',ax=axes[0],data=data3)
sns.boxplot(x='Item_Type',y='Item_Outlet_Sales',ax=axes[1],data=data3)
sns.boxplot(x='Outlet_Identifier',y='Item_Outlet_Sales',ax=axes[2],data=data3)

In [ ]:
fig,axes=plt.subplots(2,2,figsize=(15,12))
sns.boxplot(x='Outlet_Establishment_Year',y='Item_Outlet_Sales',ax=axes[0,0],data=data3)
sns.boxplot(x='Outlet_Size',y='Item_Outlet_Sales',ax=axes[0,1],data=data3)
sns.boxplot(x='Outlet_Location_Type',y='Item_Outlet_Sales',ax=axes[1,0],data=data3)
sns.boxplot(x='Outlet_Type',y='Item_Outlet_Sales',ax=axes[1,1],data=data3)

**AS MODEL CAN TAKE INTEGERS AS INPUT, WE CAN DO LABEL ENCODING FOR THE COLUMNS CONTAINING STRING VALUES**

In [ ]:
from sklearn.preprocessing import LabelEncoder 
  
le = LabelEncoder() 
le1=  LabelEncoder()
data3['Item_Fat_Content']= le.fit_transform(data3['Item_Fat_Content']) 
data3['Item_Type']=le.fit_transform(data3['Item_Type'])
data3['Outlet_Identifier']=le.fit_transform(data3['Outlet_Identifier'])
data3['Outlet_Location_Type']=le.fit_transform(data3['Outlet_Location_Type'])
data3['Outlet_Type']=le.fit_transform(data3['Outlet_Type'])
data3['Item_Identifier']= le.fit_transform(data3['Item_Identifier']) 
data3['Outlet_Size']= le1.fit_transform(data3['Outlet_Size']) 

**HEATMAP**

In [ ]:
plt.figure(figsize=(16,6))
sns.heatmap(data3.corr(), annot=True)

In [ ]:
data3

In [ ]:
y=data3['Item_Outlet_Sales']
y

In [ ]:
x=data3.drop(columns=['Item_Outlet_Sales'])
x

**IMPACT OF Item_Fat_Content ON OUTLET SALES**

In [ ]:
Outlet_Type_pivot = data3.pivot_table(index='Item_Fat_Content', values="Item_Outlet_Sales", aggfunc=np.sum)
Outlet_Type_pivot.plot(kind='bar', color='GREEN',figsize=(5,5))
plt.xlabel("Item_Fat_Content ")
plt.ylabel("Item_Outlet_Sales")
plt.title("Impact of Item_Fat_Content on Item_Outlet_Sales")
plt.xticks(rotation=0)
plt.ticklabel_format(axis="y", style="plain")
plt.show()

**IMPACT OF OUTLET TYPE ON SALES**

In [ ]:
plt.figure(figsize=(10,5))
type2=data3.groupby(['Outlet_Type'])['Item_Outlet_Sales'].sum()
store_types=['Grocery Store', 'Supermarket Type1', 'Supermarket Type2', 'Supermarket Type3']
plt.bar(store_types,type2, width=0.3,color='BLUE')
plt.ticklabel_format(axis="y", style="plain")
plt.xlabel('Outlet_Type')
plt.ylabel('Item_Outlet_Sales')
plt.title('Outlet Type vs Outlet Sales')
plt.show()

**IMPACT OF OUTLET SIZE ON OUTLET SALES**

In [ ]:
plt.figure(figsize=(10,5))
type3 = data3.groupby(['Outlet_Size'])['Item_Outlet_Sales'].sum()
size = ['High', 'Medium', 'Small']
plt.bar(size, type3, color='RED',width=0.3)
plt.ticklabel_format(axis="y", style="plain")
plt.xlabel('Outlet_Size')
plt.ylabel('Item_Outlet_Sales')
plt.title('Outlet Type vs Outlet Sales')
plt.show()

**SPLITTING OF TRAIN AND TEST DATA WITH TEST SIZE OF 25%**

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test=train_test_split(x,y, test_size=0.25)

In [ ]:
X_train.shape , X_test.shape

**FIRTS WE USE LINEAR REGRESSION MODEL**

In [ ]:
from sklearn.linear_model import LinearRegression
lr=LinearRegression()
lr.fit(X_train,Y_train)

In [ ]:
from sklearn.metrics import mean_squared_error
predict_lr=lr.predict(X_test)
mse=mean_squared_error(Y_test,predict_lr)
lr_score=np.sqrt(mse)
lr_score

**RIDGE**

In [ ]:
from sklearn.linear_model import Ridge
r=Ridge(alpha=0.05,solver='cholesky')
r.fit(X_train,Y_train)
predict_r=r.predict(X_test)
mse=mean_squared_error(Y_test,predict_r)
r_score=np.sqrt(mse)
r_score

**DECISION TREE REGRESSOR**

In [ ]:
from sklearn.tree import DecisionTreeRegressor
dtr=DecisionTreeRegressor()
dtr.fit(X_train,Y_train)
predict_r=dtr.predict(X_test)
mse=mean_squared_error(Y_test,predict_r)
dtr_score=np.sqrt(mse)
dtr_score

**RANDOM FOREST REGRESSOR**

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf=RandomForestRegressor()
rf.fit(X_train,Y_train)
predict_r=rf.predict(X_test)
mse=mean_squared_error(Y_test,predict_r)
rf_score=np.sqrt(mse)
rf_score

**GRADIENT BOOSTING REGRESSOR**

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
gbr=GradientBoostingRegressor()
gbr.fit(X_train,Y_train)
p=gbr.predict(X_test)
gb_score=mean_squared_error(Y_test,p)
gb_score=np.sqrt(gb_score)
gb_score

**WE SORT UP THE RMSE VALUES OF ALL THE MODELS USED**

In [ ]:
name=['Linear Regression','Ridge Regression',
     'Decision Tree Regression','Random Forest',
     'Gradient Boost']

In [ ]:
go=pd.DataFrame({'RMSE':[lr_score,r_score,dtr_score,rf_score,gb_score]},index=name)

In [ ]:

go.RMSE.sort_values()

**FROM ABOVE WE FIND OUT THAT GRADIEN BOOSTING REGRESSOR GIVES THE LOWEST RMSE**

**WE PLOT REGRESSION LINE **

In [ ]:
sns.regplot(Y_test, p)

**NOW WE APPLY THE MODEL TO THE TSET DATA PROVIDED BUT WE NEED TO DO ALL FORMS OF PREPROCESSING AS WE DID IN THE TRAIN DATA PROVIDED**

In [ ]:
data1.isnull().sum()

In [ ]:
data1.Item_Fat_Content=data1.Item_Fat_Content.replace('LF','Low Fat')
data1.Item_Fat_Content=data1.Item_Fat_Content.replace('reg','Regular')
data1.Item_Fat_Content=data1.Item_Fat_Content.replace('low fat','Low Fat')
data1['Item_Weight'].fillna(data1['Item_Weight'].mean(), inplace=True)
data1['Outlet_Size']=data1['Outlet_Size'].replace(np.nan,'Medium' )
data1.isnull().sum()

In [ ]:
data1['Item_Fat_Content']= le.fit_transform(data1['Item_Fat_Content']) 
data1['Item_Type']=le.fit_transform(data1['Item_Type'])
data1['Outlet_Identifier']=le.fit_transform(data1['Outlet_Identifier'])
data1['Outlet_Location_Type']=le.fit_transform(data1['Outlet_Location_Type'])
data1['Outlet_Type']=le.fit_transform(data1['Outlet_Type'])
data1['Item_Identifier']= le.fit_transform(data1['Item_Identifier']) 
data1['Outlet_Size']= le1.fit_transform(data1['Outlet_Size']) 

In [ ]:
data1

In [ ]:
predict=gbr.predict(data1)
predict

In [ ]:
sample=pd.read_csv('../input/big-mart-sales-prediction/Submission.csv')

In [ ]:
sample.head()

In [ ]:
del sample['Item_Outlet_Sales']

In [ ]:
df=pd.DataFrame({'Item_Outlet_Sales':predict})
corr_ans=pd.concat([sample,df],axis=1)
del corr_ans['Unnamed: 0']
corr_ans

In [ ]:
submit=corr_ans.to_csv('correct.csv',index=None)